In [2]:
import pandas as pd
import numpy as np
import random
import statistics
import math
from dateutil import parser
from datetime import datetime
from functools import reduce
import random
from random import choices

In [401]:
If we forward search one element at a time, most of covariances are wasted.

problem with directs aggregation: correlation is not preserved. Need finer granularity

print("hi")

hi


In [343]:
def is_numeric(df, att, impute_rate, cardinality):
    col = pd.to_numeric(df[att],errors="coerce")
    nan_count = sum(np.isnan(col))
    unique_count = len(col.unique())
    if nan_count/len(df) < impute_rate and unique_count/len(df) < cardinality:
        mean_value = col.mean()
        col.fillna(value=mean_value, inplace=True)
        return True, col
    else:
        return False, col

In [810]:
crime["Major N"]

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       1.000000
          ...   
1914    1.000000
1915    0.387234
1916    1.000000
1917    1.000000
1918    4.000000
Name: Major N, Length: 1919, dtype: float64

In [765]:
gender_crime = pd.merge(gender, crime, left_on="DBN", right_on="DBN")
gender_crime[["Mean Scale Score", "Register"]]

,Mean Scale Score,Register
0,285.0,165.0
1,292.0,165.0
2,NaN,165.0
3,NaN,165.0
4,289.0,165.0
...,...,...
43411,274.0,348.0
43412,286.0,348.0
43413,282.0,348.0
43414,289.0,348.0


In [734]:
crimedata.covariance["cov:Q:crime:Register,crime:Register"]

591300.1507974287

In [760]:
(gender_crime["Register"]*gender_crime["Register"]).sum()

21043169495.61445

In [687]:
gender_crime[["Mean Scale Score", "Register"]]

,Mean Scale Score,Register
0,285.0,165.0
1,292.0,165.0
2,NaN,165.0
3,NaN,165.0
4,289.0,165.0
...,...,...
43411,274.0,348.0
43412,286.0,348.0
43413,282.0,348.0
43414,289.0,348.0


In [1170]:
def normalize(cov):
    cols = []
    
    if isinstance(cov, pd.DataFrame):
        cols = cov.columns
    else:
        cols = list(cov.axes[0])
        
    for col in cols:
        if col != 'cov:c':
            cov[col] = cov[col]/cov['cov:c']
    cov['cov:c'] = 1
    return cov

class agg_dataset:
    def set_meta(self, data, X, dimensions, name):
        self.data = data
        self.dimensions = dimensions
        self.X = X
        self.name = name
        
    def load_buyer(self, data, X, y, dimensions, name):
        self.data = data
        self.dimensions = dimensions
        self.X = X
        self.y = y
        self.name = name
        
        # don't impute y
        self.to_numeric(self.y, False)
        for x in self.X:
            self.to_numeric(x, False)
        
        self.X =  [self.y] + self.X
        
        dedup_dimensions = set()
        for d in dimensions:
            if isinstance(d, list):
                dedup_dimensions.update(d)
            else:
                dedup_dimensions.add(d)
        dedup_dimensions = list(dedup_dimensions)
        
        # project out attributes except x, y, dim
        self.data = self.data[self.X  + dedup_dimensions]
        
        
    
    def compute_agg(self, norm = False):
        self.lift(self.name, self.X)
        
        self.agg_dimensions = dict()
        
        for d in self.dimensions:
            if isinstance(d, list):
                self.agg_dimensions[tuple(d)] = self.data[list(filter(lambda col: col.startswith("cov:"), self.data.columns)) + d].groupby(d).sum()
            else:
                self.agg_dimensions[d] = self.data[list(filter(lambda col: col.startswith("cov:"), self.data.columns)) + [d]].groupby(d).sum()
            
        if norm:
            for d in self.agg_dimensions.keys():
                self.agg_dimensions[d] = normalize(self.agg_dimensions[d])
            
        self.covariance = normalize(self.data[list(filter(lambda col: col.startswith("cov:"), self.data.columns))].sum())
    
    def load_seller(self, data, dimensions, name):
        self.data = data
        self.dimensions = dimensions
        self.name = name
        
                
        dedup_dimensions = set()
        for d in dimensions:
            if isinstance(d, list):
                dedup_dimensions.update(d)
            else:
                dedup_dimensions.add(d)
        dedup_dimensions = list(dedup_dimensions)
        
        
        # find numeric attributes as features
        atts = []
        for att in self.data.columns:
            if att in dedup_dimensions:
                continue
            cond, col = self.is_numeric(att, 0.4, 2)
            if cond:
                self.data[att] = col
                self.data["log" + att] = np.log(self.data[att])
                self.data["sq" + att] = np.square(self.data[att])
                self.data["cbr" + att] = np.cbrt(self.data[att])
                atts.append(att)
                atts.append("log" + att)
                atts.append("sq" + att)
                atts.append("cbr" + att)

        self.X = atts

        
        # project out attributes except x, y, dim
        self.data = self.data[self.X + dedup_dimensions]

    def is_numeric(self, att, impute_rate, cardinality):
        col = pd.to_numeric(self.data[att],errors="coerce")
        nan_count = sum(np.isnan(col))
        unique_count = len(col.unique())
        if nan_count/len(self.data) < impute_rate and unique_count/len(self.data) < cardinality:
            mean_value = col.mean()
            col.fillna(value=mean_value, inplace=True)
            return True, col
        else:
            return False, col
    
    def to_numeric(self, att, impute=True, impute_rate = 1):
        # parse attribute to numeric
        self.data[att] = pd.to_numeric(self.data[att],errors="coerce")
        # count the number of nan
        nan_count = sum(np.isnan(self.data[att]))
        
        if impute:
            # impute error only if missing rate is not above threshold
            if nan_count/len(self.data) < impute_rate:
                mean_value=self.data[att].mean()
                self.data[att].fillna(value=mean_value, inplace=True)
                return True
            else:
                return False
        else:
            # else, remove records with missing value
            self.data = self.data[~np.isnan(self.data[att])]
    
    def lift(self, tablename, attributes):
        self.data['cov:c'] = 1

        for i in range(len(attributes)):
            for j in range(i, len(attributes)):
                self.data['cov:Q:' + tablename + ":" + attributes[i] + ","+ tablename + ":" + attributes[j]] = self.data[attributes[i]] * self.data[attributes[j]]

        for attribute in attributes:
            self.data= self.data.rename(columns = {attribute:'cov:s:' + tablename + ":" + attribute})

In [1141]:
len(gender_train.data) + len(gender_test.data)

47270

In [938]:
%%time


CPU times: user 28 ms, sys: 2.67 ms, total: 30.6 ms
Wall time: 28.7 ms


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [939]:
gender_train = agg_dataset()
gender_test = agg_dataset()
gender_train.load_buyer(gender, ["Number Tested"], "Mean Scale Score", ["DBN", ["DBN","Grade"], "School Name", "Year", "Category"], "gender")
msk = np.random.rand(len(gender_train.data)) < 0.8
gender_test.set_meta(gender_train.data[~msk], ["Mean Scale Score", "Number Tested"], ["DBN", ["DBN","Grade"], "School Name", "Year", "Category"], "gender")
gender_train.data = gender_train.data[msk]
gender_test.compute_agg()
gender_train.compute_agg()

In [681]:
total_sum_of_square(genderdata.agg_dimensions["DBN"].sum(), "gender:Mean Scale Score")

16228455.420372486

In [541]:
normalize(genderdata.agg_dimensions["DBN"])

cov:s:gender:Number Tested
cov:s:gender:Mean Scale Score
cov:c
cov:Q:gender:Number Tested,gender:Number Tested
cov:Q:gender:Number Tested,gender:Mean Scale Score
cov:Q:gender:Mean Scale Score,gender:Mean Scale Score


,cov:s:gender:Number Tested,cov:s:gender:Mean Scale Score,cov:c,"cov:Q:gender:Number Tested,gender:Number Tested","cov:Q:gender:Number Tested,gender:Mean Scale Score","cov:Q:gender:Mean Scale Score,gender:Mean Scale Score"
DBN,,,,,,
01M015,0.000521,0.008845,1,0.011879,0.150848,0.031250
01M019,0.000379,0.004797,1,0.012538,0.116218,0.025000
01M020,0.000877,0.004615,1,0.066520,0.258446,0.025000
01M034,0.000103,0.000861,1,0.007552,0.030445,0.014286
01M063,0.000961,0.017250,1,0.020905,0.291197,0.038462
...,...,...,...,...,...,...
32K377,0.000124,0.000817,1,0.011131,0.034548,0.014286
32K383,0.001382,0.002525,1,0.395397,0.435630,0.020000
32K384,0.000152,0.000857,1,0.016443,0.044816,0.014286


In [1087]:
%%time
crime = pd.read_csv("crime.csv")
crimedata = agg_dataset()
crimedata.load_seller(crime, ["DBN"], "crime")
crimedata.compute_agg()

/opt/conda/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:141: PerformanceWarning: DataFrame is highly fragmented.  T

CPU times: user 1.98 s, sys: 95.9 ms, total: 2.08 s
Wall time: 2.01 s


In [1115]:
crimedata.agg_dimensions["DBN"]

,cov:s:crime:ID,cov:s:crime:logID,cov:s:crime:sqID,cov:s:crime:cbrID,cov:s:crime:Geographical District Code,cov:s:crime:logGeographical District Code,cov:s:crime:sqGeographical District Code,cov:s:crime:cbrGeographical District Code,cov:s:crime:Register,cov:s:crime:logRegister,...,"cov:Q:crime:Zip Codes,crime:Zip Codes","cov:Q:crime:Zip Codes,crime:logZip Codes","cov:Q:crime:Zip Codes,crime:sqZip Codes","cov:Q:crime:Zip Codes,crime:cbrZip Codes","cov:Q:crime:logZip Codes,crime:logZip Codes","cov:Q:crime:logZip Codes,crime:sqZip Codes","cov:Q:crime:logZip Codes,crime:cbrZip Codes","cov:Q:crime:sqZip Codes,crime:sqZip Codes","cov:Q:crime:sqZip Codes,crime:cbrZip Codes","cov:Q:crime:cbrZip Codes,crime:cbrZip Codes"
DBN,,,,,,,,,,,,,,,,,,,,,
01M015,641,6.463029,410881,8.622225,1,0.000000,1,1.000000,165.0,5.105945,...,164326761,121250.866662,2106504749259,300012.163880,89.466698,1.554315e+09,221.368295,27003284380751120,3.845856e+09,547.733661
01M019,645,6.469250,416025,8.640123,1,0.000000,1,1.000000,254.0,5.537334,...,597753601,247041.120985,14614477790849,709601.950729,102.097780,6.039908e+09,293.266090,357309367508467200,1.734906e+10,842.378745
01M020,648,6.473891,419904,8.653497,1,0.000000,1,1.000000,463.0,6.137727,...,285069456,164350.910899,4813112695104,433145.484529,94.753125,2.774901e+09,249.721089,81264594744135936,7.313228e+09,658.137892
01M034,660,6.492240,435600,8.706588,1,0.000000,1,1.000000,320.0,5.768321,...,167832025,122673.966540,2174263883875,304263.523142,89.666451,1.589241e+09,222.396252,28167588615600624,3.941734e+09,551.600873
01M063,699,6.549651,488601,8.874810,1,0.000000,1,1.000000,182.0,5.204007,...,955119025,319516.697165,29517953467625,969850.953794,106.888165,9.874664e+09,324.444981,912252351916950656,2.997324e+10,984.810111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85X157,1738,7.460490,3020644,12.023104,85,4.442651,7225,4.396830,587.0,6.375025,...,416649744,202566.202679,8504654574528,557845.686845,98.483359,4.134781e+09,271.212653,173597009175265536,1.138675e+10,746.890679
85X195,1590,7.371489,2528100,11.671653,12,2.484907,144,2.289428,911.0,6.814543,...,134792100,108665.212287,1564936281000,262891.347807,87.602525,1.261603e+09,211.934855,18168910222410000,3.052169e+09,512.729312
85X196,1591,7.372118,2531281,11.674100,12,2.484907,144,2.289428,907.0,6.810142,...,134792100,108665.212287,1564936281000,262891.347807,87.602525,1.261603e+09,211.934855,18168910222410000,3.052169e+09,512.729312


In [1112]:
%%time
esl = pd.read_csv("esl.csv")
esldata = agg_dataset()
esldata.load_seller(esl, [["DBN","Grade"]], "esl")
esldata.compute_agg(True)

/opt/conda/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:149: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


CPU times: user 5.97 s, sys: 3.85 s, total: 9.83 s
Wall time: 9.82 s


In [1123]:
esl

,DBN,Grade,Year,Category,Number Tested,Mean Scale Score,Level 1 #,Level 1 %,Level 2 #,Level 2 %,...,cbrLevel 4 #,logLevel 4 %,sqLevel 4 %,cbrLevel 4 %,logLevel 3+4 #,sqLevel 3+4 #,cbrLevel 3+4 #,logLevel 3+4 %,sqLevel 3+4 %,cbrLevel 3+4 %
0,01M015,3,2006,All Students,37,651.000000,4.0,10.8,14.0,37.8,...,0.000000,-inf,0.00,0.000000,2.944439,361.0,2.668402,3.939638,2641.96,3.718100
1,01M015,4,2006,All Students,46,621.000000,18.0,39.1,20.0,43.5,...,1.000000,0.788457,4.84,1.300591,2.079442,64.0,2.000000,2.856470,302.76,2.591292
2,01M015,5,2006,All Students,28,625.000000,8.0,28.6,13.0,46.4,...,0.000000,-inf,0.00,0.000000,1.945910,49.0,1.912931,3.218876,625.00,2.924018
3,01M015,6,2006,All Students,36,639.000000,2.0,5.6,18.0,50.0,...,1.000000,1.029619,7.84,1.409460,2.772589,256.0,2.519842,3.793239,1971.36,3.541014
4,01M015,All Grades,2006,All Students,147,656.794021,32.0,21.8,65.0,44.2,...,1.259921,0.336472,1.96,1.118689,3.912023,2500.0,3.684031,3.526361,1156.00,3.239612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28460,75X723,5,2011,All Students,55,634.000000,40.0,72.7,13.0,23.6,...,0.000000,-inf,0.00,0.000000,0.693147,4.0,1.259921,1.280934,12.96,1.532619
28461,75X723,6,2011,All Students,54,623.000000,44.0,81.5,10.0,18.5,...,0.000000,-inf,0.00,0.000000,-inf,0.0,0.000000,-inf,0.00,0.000000
28462,75X723,7,2011,All Students,28,613.000000,21.0,75.0,7.0,25.0,...,0.000000,-inf,0.00,0.000000,-inf,0.0,0.000000,-inf,0.00,0.000000
28463,75X723,8,2011,All Students,32,587.000000,27.0,84.4,5.0,15.6,...,0.000000,-inf,0.00,0.000000,-inf,0.0,0.000000,-inf,0.00,0.000000


In [566]:
crimedata.agg_dimensions["DBN"]

,cov:s:crime:ID,cov:s:crime:Geographical District Code,cov:s:crime:Register,cov:s:crime:# Schools,cov:s:crime:Major N,cov:s:crime:Oth N,cov:s:crime:NoCrim N,cov:s:crime:Prop N,cov:s:crime:Vio N,cov:s:crime:AvgOfMajor N,...,"cov:Q:crime:AvgOfNoCrim N,crime:AvgOfNoCrim N","cov:Q:crime:AvgOfNoCrim N,crime:AvgOfProp N","cov:Q:crime:AvgOfNoCrim N,crime:AvgOfVio N","cov:Q:crime:AvgOfNoCrim N,crime:Zip Codes","cov:Q:crime:AvgOfProp N,crime:AvgOfProp N","cov:Q:crime:AvgOfProp N,crime:AvgOfVio N","cov:Q:crime:AvgOfProp N,crime:Zip Codes","cov:Q:crime:AvgOfVio N,crime:AvgOfVio N","cov:Q:crime:AvgOfVio N,crime:Zip Codes","cov:Q:crime:Zip Codes,crime:Zip Codes"
DBN,,,,,,,,,,,,,,,,,,,,,
01M015,641.0,1.0,165.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,...,2.856100,0.96330,0.760500,21664.110000,0.32490,0.256500,7306.830000,0.202500,5768.550000,164326761.0
01M019,645.0,1.0,254.0,2.0,0.387234,1.698723,4.169362,0.916596,0.929362,0.385263,...,17.064158,3.73889,3.819835,100995.849742,0.81922,0.836956,22128.977321,0.855075,22608.059793,597753601.0
01M020,648.0,1.0,463.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,...,4.928400,1.22100,1.287600,37482.480000,0.30250,0.319000,9286.200000,0.336400,9792.720000,285069456.0
01M034,660.0,1.0,320.0,1.0,0.000000,0.000000,7.000000,0.000000,0.000000,0.250000,...,4.928400,1.22100,1.287600,28760.100000,0.30250,0.319000,7125.250000,0.336400,7513.900000,167832025.0
01M063,699.0,1.0,182.0,2.0,0.387234,1.698723,4.169362,0.916596,0.929362,0.385263,...,17.064158,3.73889,3.819835,127664.801680,0.81922,0.836956,27972.352412,0.855075,28577.941344,955119025.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85X157,1738.0,85.0,587.0,1.0,0.000000,1.000000,0.000000,1.000000,0.000000,0.260000,...,5.808100,1.59060,1.373700,49192.920000,0.43560,0.376200,13471.920000,0.324900,11634.840000,416649744.0
85X195,1590.0,12.0,911.0,2.0,0.387234,1.698723,4.169362,0.916596,0.929362,0.385263,...,17.064158,3.73889,3.819835,47959.500000,0.81922,0.836956,10508.300000,0.855075,10735.800000,134792100.0
85X196,1591.0,12.0,907.0,2.0,0.387234,1.698723,4.169362,0.916596,0.929362,0.385263,...,17.064158,3.73889,3.819835,47959.500000,0.81922,0.836956,10508.300000,0.855075,10735.800000,134792100.0


In [477]:
parameter = linear_regression(crimedata.covariance, ["ID"], "Register", "crime")
r2(crimedata.covariance, ["ID"], "Register", parameter, "crime")

0.00270470284023272

In [1134]:
# return the coefficients of features and a constant 
def linear_regression(cov_matrix, features, result):
    a = np.empty([len(features) + 1, len(features) + 1])
    b = np.empty(len(features) + 1)
    
    for i in range(len(features)):
        for j in range(len(features)):
            if 'cov:Q:' + features[i] + ","+ features[j] in cov_matrix:
                a[i][j] = cov_matrix['cov:Q:' + features[i] + ","+ features[j]]
            else:
                a[i][j] = cov_matrix['cov:Q:' + features[j] + ","+ features[i]]
    
    for i in range(len(features)):
        a[i][len(features)] = cov_matrix['cov:s:' + features[i]]
        a[len(features)][i] = cov_matrix['cov:s:' + features[i]]
        if 'cov:Q:' + result + "," + features[i] in cov_matrix:
            b[i] = cov_matrix['cov:Q:' + result + "," + features[i]]
        else:
            b[i] = cov_matrix['cov:Q:' + features[i] + "," + result]
    
    b[len(features)] = cov_matrix['cov:s:' + result]
    
    a[len(features)][len(features)] = cov_matrix['cov:c']
    print(a,b)
    return np.linalg.solve(a, b)

def square_error(cov_matrix, features, result, parameter):
    se = cov_matrix['cov:Q:'  + result + "," + result]
    
    for i in range(len(features)):
        for j in range(len(features)):
            if 'cov:Q:'  + features[i] + "," + features[j] in cov_matrix:
                se += parameter[min(i,j)] * parameter[max(i,j)] * cov_matrix['cov:Q:'  + features[i] + "," + features[j]]
            else:    
                se += parameter[min(i,j)] * parameter[max(i,j)] * cov_matrix['cov:Q:'  + features[j] + "," + features[i]]
    
    for i in range(len(features)):
        se += 2 * parameter[i] * parameter[-1] * cov_matrix['cov:s:'  + features[i]]
        if 'cov:Q:' + result + "," + features[i] in cov_matrix:
            se -= 2 * parameter[i] *  cov_matrix['cov:Q:' + result + "," + features[i]]
        else:
            se -= 2 * parameter[i] *  cov_matrix['cov:Q:' + features[i] + "," + result]

    se -= 2 * parameter[-1] * cov_matrix['cov:s:'  + result]
    se += cov_matrix['cov:c'] * parameter[-1] * parameter[-1]

    return se

def total_sum_of_square(cov_matrix, result):
    return cov_matrix['cov:Q:'  + result + "," + result] - cov_matrix['cov:s:'  + result] * cov_matrix['cov:s:'  + result] / cov_matrix['cov:c']

def mean_squared_error(cov_matrix, features, result, parameter):
    return square_error(cov_matrix, features, result, parameter)/cov_matrix['cov:c']


def r2(cov_matrix, features, result, parameter):
    return 1 - square_error(cov_matrix, features, result, parameter)/total_sum_of_square(cov_matrix, result)

def adjusted_r2(cov_matrix, features, result, parameter):
    return 1 - (cov_matrix['cov:c']-1)*(1 - r2(cov_matrix, features, result, parameter))/(cov_matrix['cov:c'] - len(parameter) - 1)

In [799]:
parameter = linear_regression(genderdata.covariance, ["gender:Number Tested"], "gender:Mean Scale Score")
r2(genderdata.covariance, ["gender:Number Tested"], "gender:Mean Scale Score", parameter)

90035.73300190395
cov:Q:gender:Number Tested,gender:Number Tested
90041.77147420504


0.0099188444284658

In [448]:
tuple([2,4])

(2, 4)

In [441]:
crime = pd.read_csv("crime.csv")

In [597]:
crimedata.covariance['cov:s:' + right_tablename + ":" + att2]

NameError: name 'right_tablename' is not defined

In [1164]:
def connect(aggdata1, aggdata2, dimension, left_inp = False):
    if isinstance(dimension, list):
        dimension = tuple(dimension)
    
    if left_inp:
        agg1 = aggdata1.data
    else:
        agg1 = aggdata1.agg_dimensions[dimension]
    agg2 = aggdata2.agg_dimensions[dimension]
    
    if left_inp:
        join = pd.merge(agg1, agg2, how='left', left_on=dimension, right_index=True)
    else:
        join = pd.merge(agg1, agg2, how='left', left_index=True, right_index=True)
#         join = pd.merge(agg1, agg2, how='inner', left_index=True, right_index=True)
    join = join.drop('cov:c_y', 1)
    join = join.rename(columns = {'cov:c_x':'cov:c'})
    
    left_attributes = aggdata1.X
    left_tablename = aggdata1.name
    right_attributes = aggdata2.X
    right_tablename = aggdata2.name
    
    right_cov = aggdata2.covariance
    
    # fill in nan
    for att2 in right_attributes:
        join['cov:s:' + right_tablename + ":" + att2].fillna(value=right_cov['cov:s:' + right_tablename + ":" + att2], inplace=True)
        join['cov:s:' + right_tablename + ":" + att2] *= join['cov:c']
    
    for i in range(len(right_attributes)):
        for j in range(i, len(right_attributes)):
            join['cov:Q:' + right_tablename + ":" + right_attributes[i] + ","+ right_tablename + ":" + right_attributes[j]].fillna(value=right_cov['cov:Q:' + right_tablename + ":" + right_attributes[i] + ","+ right_tablename + ":" + right_attributes[j]], inplace=True)
            join['cov:Q:' + right_tablename + ":" + right_attributes[i] + ","+ right_tablename + ":" + right_attributes[j]] *= join['cov:c']
    
    
    for att1 in left_attributes:
        for att2 in right_attributes:
            join['cov:Q:' + left_tablename + ":" + att1 + ","+ right_tablename + ":" + att2] = join['cov:s:' + left_tablename + ":" + att1] * join['cov:s:' + right_tablename + ":" + att2]/join['cov:c']
    
    return join



In [1152]:
# join = connect(genderdata, crimedata, "DBN")
# join_test = connect(gender_test, crimedata, "DBN")
# join_train = connect(gender_train, crimedata, "DBN")

join_test = connect(gender_test, esldata, ("DBN","Grade"))
join_train = connect(gender_train, esldata, ("DBN","Grade"))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app.launch_new_instance()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


3161454616.0

In [1029]:
join_test2 = connect(gender_test, crimedata, "DBN", True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app.launch_new_instance()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [1045]:
join_test2.sum()

cov:s:gender:Mean Scale Score                                                             2832625.0
cov:s:gender:Number Tested                                                                   787156
Grade                                             33445All GradesAll Grades344455All Grades33345...
School Name                                       PS 015 ROBERTO CLEMENTEPS 015 ROBERTO CLEMENTE...
Year                                                                                       19080066
                                                                        ...                        
cov:Q:gender:Number Tested,crime:cbrAvgOfVio N                                        776153.340146
cov:Q:gender:Number Tested,crime:Zip Codes                                       11972904290.732672
cov:Q:gender:Number Tested,crime:logZip Codes                                        7442287.943753
cov:Q:gender:Number Tested,crime:sqZip Codes                                  223520525452474.34375


In [1000]:
# gender_test.data

In [790]:
square_error(join.sum(), ["gender:Number Tested","crime:Register", "crime:Major N"], "gender:Mean Scale Score", parameter)

4255989099.0


nan

In [1168]:
(join_train.sum() + join_test.sum())["cov:Q:gender:Mean Scale Score,gender:Mean Scale Score"]

3955320260.0

In [1156]:
# join.sum()

parameter = linear_regression(join_train.sum() + join_test.sum(), ["esl:Mean Scale Score"], "gender:Mean Scale Score")
# square_error(join.sum(), ["crime:Register"], "gender:Mean Scale Score", parameter)
# total_sum_of_square(join.sum(), "gender:Mean Scale Score")
r2(join_train.sum() + join_test.sum(), ["esl:Mean Scale Score"], "gender:Mean Scale Score", parameter)

[[1.90717136e+10 2.89309119e+07]
 [2.89309119e+07 4.39070000e+04]] [8.67382037e+09 1.31534600e+07]


0.35861471904807185

In [1074]:
kept_cols = []
for col in crimedata.data.columns:
    names = col[6:].split(",")
    match = True
    for name in names:
        print(name)
        if name not in cur_atts:
            match = False
    if match:
        kept_cols.append(col)


In [1075]:
kept_cols

['cov:s:crime:ID',
 'cov:s:crime:sqID',
 'cov:s:crime:logRegister',
 'cov:s:crime:cbrNoCrim N',
 'cov:Q:crime:ID,crime:ID',
 'cov:Q:crime:ID,crime:sqID',
 'cov:Q:crime:ID,crime:logRegister',
 'cov:Q:crime:ID,crime:cbrNoCrim N',
 'cov:Q:crime:sqID,crime:sqID',
 'cov:Q:crime:sqID,crime:logRegister',
 'cov:Q:crime:sqID,crime:cbrNoCrim N',
 'cov:Q:crime:logRegister,crime:logRegister',
 'cov:Q:crime:logRegister,crime:cbrNoCrim N',
 'cov:Q:crime:cbrNoCrim N,crime:cbrNoCrim N']

In [1055]:
cur_atts

['crime:logRegister', 'crime:cbrNoCrim N', 'crime:sqID', 'crime:ID']

In [1050]:
cur_atts = []
join_train_cov = join_train.sum()
join_test_cov = join_test.sum()

for i in range(4):
    best_r2 = 0
    best_att = -1
    for att in ['gender:Number Tested'] + ['crime:'+x for x in crimedata.X]:
        if att in cur_atts:
            continue
        parameter = linear_regression(join_train_cov, cur_atts + [att], "gender:Mean Scale Score")
        cur_r2 = r2(join_test_cov, cur_atts + [att], "gender:Mean Scale Score", parameter)
#         print(cur_r2, att)
        if cur_r2 > best_r2:
            best_r2 = cur_r2
            best_att = att
    cur_atts = cur_atts + [best_att]
    print(i, best_r2, cur_atts)

0 0.07831764774336725 ['crime:logRegister']
1 0.11766014133624592 ['crime:logRegister', 'crime:cbrNoCrim N']
2 0.14281875887330497 ['crime:logRegister', 'crime:cbrNoCrim N', 'crime:sqID']
3 0.17317620612298834 ['crime:logRegister', 'crime:cbrNoCrim N', 'crime:sqID', 'crime:ID']


In [980]:
join.sum()["cov:Q:gender:Number Tested,crime:Register"]

3097468817.9411135

In [978]:
len(['crime:'+x for x in crimedata.X])

60

47270.0

In [748]:
square_error(join.sum(), ["crime:Register"], "gender:Mean Scale Score", parameter)

3893540297.0
3893551916.1205573
3893528677.879442
-3941965312.7074385
15062176.303714275


15062176.303714275

In [749]:
total_sum_of_square(join.sum(), "gender:Mean Scale Score")

14683867.07755375

In [759]:
parameter

array([1.47877339e-02, 2.91184167e+02])

In [764]:
join["cov:c"].sum()

43054

In [668]:
join.sum()["cov:Q:gender:Mean Scale Score,gender:Mean Scale Score"]

4255989099.0

In [615]:
'cov:Q:gender:Number Tested,crime:Register' in join.sum().axes[0]

True

In [240]:
%%time
crime["cov"] = crime.apply(lambda row: cov.lift(row, "crime", atts), axis=1)


CPU times: user 3.04 s, sys: 1.58 ms, total: 3.04 s
Wall time: 3.04 s


In [260]:
%%time
lift(crime, "crime", atts)

CPU times: user 58.6 ms, sys: 4.58 ms, total: 63.2 ms
Wall time: 58.4 ms


In [319]:
%%time
crime.sum()

CPU times: user 21.7 ms, sys: 218 µs, total: 22 ms
Wall time: 20.1 ms


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


ID                                                                                   1868188
Building Code                              K247K281K298K318K327K369K410K728M020M171M540M8...
Location Name                              PS 247(BROOKLYN)IS 281(BS)PS 298(BROOKLYN)JHS ...
Address                                    7000 21st AVENUE8787  24th AVENUE85 WATKINS ST...
Borough                                    KKKKKKKKMMMMQQRRXXXXXXXXXXMMKQMMMKKQQMKKQKQMMM...
                                                                 ...                        
cov:Q:crime:AvgOfProp Ncrime:AvgOfVio N                                          2076.527968
cov:Q:crime:AvgOfProp Ncrime:Zip Codes                                       26021772.606313
cov:Q:crime:AvgOfVio Ncrime:AvgOfVio N                                           2166.171713
cov:Q:crime:AvgOfVio Ncrime:Zip Codes                                        26600008.255375
cov:Q:crime:Zip Codescrime:Zip Codes                                  

In [254]:
%%time
crime["ID"] * crime["Geographical District Code"]

CPU times: user 340 µs, sys: 0 ns, total: 340 µs
Wall time: 325 µs


0        5760
1        7035
2        8257
3        5460
4        9200
        ...  
1914    14128
1915    12061
1916    13056
1917    13491
1918    39556
Length: 1919, dtype: int64

In [211]:
crime.groupby("DBN").agg({"cov": lambda series: reduce(lambda x, y: cov.add(x, y), series)})

,cov
DBN,
01M015,"{'c': 1, 's': {'crime:ID': 641, 'crime:Geograp..."
01M019,"{'c': 1, 's': {'crime:ID': 645, 'crime:Geograp..."
01M020,"{'c': 1, 's': {'crime:ID': 648, 'crime:Geograp..."
01M034,"{'c': 1, 's': {'crime:ID': 660, 'crime:Geograp..."
01M063,"{'c': 1, 's': {'crime:ID': 699, 'crime:Geograp..."
...,...
85X157,"{'c': 1, 's': {'crime:ID': 1738, 'crime:Geogra..."
85X195,"{'c': 1, 's': {'crime:ID': 1590, 'crime:Geogra..."
85X196,"{'c': 1, 's': {'crime:ID': 1591, 'crime:Geogra..."


In [215]:
join = pd.merge(genderdata.agg_dimensions["DBN"], crime.groupby("DBN").agg({"cov": lambda series: reduce(lambda x, y: cov.add(x, y), series)}), left_index=True, right_index=True)

In [217]:
join["cov"] = join.apply(lambda row: cov.multiply(row["cov_x"], row["cov_y"] ), axis=1)

In [507]:
# cov_matrix = join.agg({"cov": lambda series: reduce(lambda x, y: cov.add(x, y), series)}).iloc[0]

0.08091140780075012 crime:Register


In [332]:
parameter = linear_regression(cov_matrix, ['crime:Register', 'gender:Number Tested', 'crime:# Schools', 'crime:AvgOfMajor N', 'crime:NoCrim N', 'crime:ID', 'crime:AvgOfNoCrim N', 'crime:AvgOfOth N', 'crime:Zip Codes', 'crime:AvgOfVio N', 'crime:Geographical District Code', 'crime:AvgOfProp N'], "gender:Mean Scale Score")
r2(cov_matrix, ['crime:Register', 'gender:Number Tested', 'crime:# Schools', 'crime:AvgOfMajor N', 'crime:NoCrim N', 'crime:ID', 'crime:AvgOfNoCrim N', 'crime:AvgOfOth N', 'crime:Zip Codes', 'crime:AvgOfVio N', 'crime:Geographical District Code', 'crime:AvgOfProp N'], "gender:Mean Scale Score", parameter)

0.13218764212317957

In [167]:
%%time
join = pd.merge(gender, crime, how='inner', left_on = 'DBN', right_on = 'DBN')

CPU times: user 55.2 ms, sys: 3.75 ms, total: 59 ms
Wall time: 56.5 ms


In [161]:
gender.groupby(["DBN", "Year"]).mean()

Number Tested  Mean Scale Score
DBN    Year                                 
01M015 2013      17.750000        283.250000
       2014      15.750000        277.333333
       2015      14.000000        282.333333
       2016      12.750000        295.000000
       2017      16.750000        308.875000
...                    ...               ...
32K562 2013      71.500000        272.500000
       2014      74.666667        271.166667
       2015      77.250000        276.250000
       2016      66.500000        284.000000
       2017      70.500000        287.375000

[5543 rows x 2 columns]

In [136]:
gender = pd.read_csv("gender.csv")
gender

,DBN,School Name,Grade,Year,Category,Number Tested,Mean Scale Score,Level1_N,Level1_%,Level2_N,Level2_%,Level3_N,Level3_%,Level4_N,Level4_%,Level3+4_N,Level3+4_%
0,01M015,PS 015 ROBERTO CLEMENTE,3,2013,Female,12,285,7,58.3,5,41.7,0,0.0,0,0.0,0,0.0
1,01M015,PS 015 ROBERTO CLEMENTE,3,2013,Male,15,292,7,46.7,6,40.0,2,13.3,0,0.0,2,13.3
2,01M015,PS 015 ROBERTO CLEMENTE,3,2014,Female,13,s,s,s,s,s,s,s,s,s,s,s
3,01M015,PS 015 ROBERTO CLEMENTE,3,2014,Male,5,s,s,s,s,s,s,s,s,s,s,s
4,01M015,PS 015 ROBERTO CLEMENTE,3,2015,Female,7,289,3,42.9,3,42.9,1,14.3,0,0.0,1,14.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47730,32K562,EVERGREEN MIDDLE SCHOOL FOR URBAN EXPLORATION,All Grades,2015,Male,161,274,98,60.9,52,32.3,11,6.8,0,0.0,11,6.8
47731,32K562,EVERGREEN MIDDLE SCHOOL FOR URBAN EXPLORATION,All Grades,2016,Female,125,286,48,38.4,63,50.4,12,9.6,2,1.6,14,11.2
47732,32K562,EVERGREEN MIDDLE SCHOOL FOR URBAN EXPLORATION,All Grades,2016,Male,141,282,62,44.0,70,49.6,9,6.4,0,0.0,9,6.4
47733,32K562,EVERGREEN MIDDLE SCHOOL FOR URBAN EXPLORATION,All Grades,2017,Female,138,289,49,35.5,70,50.7,14,10.1,5,3.6,19,13.8


In [102]:
gender["Mean Scale Score"] = pd.to_numeric(gender["Mean Scale Score"],errors="coerce")


In [105]:
sum(np.isnan(gender["Mean Scale Score"]))

465

In [18]:
gender[["Mean Scale Score", "Number Tested"]]

,Mean Scale Score,Number Tested
0,285.0,12
1,292.0,15
4,289.0,7
5,276.0,9
8,308.0,13
...,...,...
47730,274.0,161
47731,286.0,125
47732,282.0,141
47733,289.0,138


In [23]:
%%time
# slow because dataframe apply copies row and apply function one by one
gender["cov"] = gender.apply(lambda row: cov.lift(row, "gender", ["Mean Scale Score", "Number Tested"]), axis=1)

CPU times: user 2.34 s, sys: 6.84 ms, total: 2.35 s
Wall time: 2.35 s


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [84]:
grade = gender.groupby("Grade").agg({"cov": lambda series: reduce(lambda x, y: cov.add(x, y), series)})

In [100]:
gender.groupby("Year").mean()

,Number Tested,Mean Scale Score
Year,,
2013,84.762075,295.774558
2014,83.330172,295.788127
2015,81.170757,296.739387
2016,79.932806,303.642879
2017,79.697036,305.237007


In [83]:
%%time
# marginalize all attributes
cov_matrix = gender.agg({"cov": lambda series: reduce(lambda x, y: cov.add(x, y), series)}).iloc[0]

CPU times: user 237 ms, sys: 0 ns, total: 237 ms
Wall time: 235 ms


In [340]:
%%time
parameter = linear_regression(cov_matrix, ["gender:Number Tested"], "gender:Mean Scale Score")
parameter

CPU times: user 230 µs, sys: 43 µs, total: 273 µs
Wall time: 279 µs


array([1.87138466e-02, 2.98600968e+02])

In [155]:
parameter = ridge_linear_regression(cov_matrix, ["gender:Number Tested"], "gender:Mean Scale Score",10000000)
parameter

array([1.93755959e-02, 2.97902986e+02])

array([1.98498731e-02, 2.97864214e+02])

In [50]:
square_error(cov_matrix, ["gender:Number Tested"], "gender:Mean Scale Score", parameter)

339.9087771470889

In [54]:
total_sum_of_square(cov_matrix, "gender:Mean Scale Score")

16228455.420372486

In [341]:
r2(cov_matrix, ["gender:Number Tested"], "gender:Mean Scale Score", parameter)

0.008981579685800378

In [342]:
adjusted_r2(cov_matrix, ["gender:Number Tested"], "gender:Mean Scale Score", parameter)

0.008935540410507592

In [56]:
1 - square_error(cov_matrix, ["gender:Number Tested"], "gender:Mean Scale Score", parameter)/total_sum_of_square(cov_matrix, "gender:Mean Scale Score")

0.009918844428504325

In [307]:
%%time
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(genderdata.data[["Number Tested","Level1_%","Level2_%","Level3_%","Level4_%"]], genderdata.data[["Mean Scale Score"]])
print(reg.coef_)
print(reg.intercept_)

[[-0.00294854 -0.79722224 -0.35483165  0.01766236  0.22090017]]
[335.96515967]
CPU times: user 34.1 ms, sys: 170 µs, total: 34.3 ms
Wall time: 32.4 ms


In [283]:
genderdata.data

,DBN,School Name,Grade,Year,Category,Number Tested,Mean Scale Score,Level1_N,Level1_%,Level2_N,Level2_%,Level3_N,Level3_%,Level4_N,Level4_%,Level3+4_N,Level3+4_%,cov
0,01M015,PS 015 ROBERTO CLEMENTE,3,2013,Female,12,285.0,7,58.3,5,41.7,0,0.0,0,0.0,0,0.0,"{'c': 1, 's': {'gender:Mean Scale Score': 285...."
1,01M015,PS 015 ROBERTO CLEMENTE,3,2013,Male,15,292.0,7,46.7,6,40.0,2,13.3,0,0.0,2,13.3,"{'c': 1, 's': {'gender:Mean Scale Score': 292...."
4,01M015,PS 015 ROBERTO CLEMENTE,3,2015,Female,7,289.0,3,42.9,3,42.9,1,14.3,0,0.0,1,14.3,"{'c': 1, 's': {'gender:Mean Scale Score': 289...."
5,01M015,PS 015 ROBERTO CLEMENTE,3,2015,Male,9,276.0,6,66.7,2,22.2,1,11.1,0,0.0,1,11.1,"{'c': 1, 's': {'gender:Mean Scale Score': 276...."
8,01M015,PS 015 ROBERTO CLEMENTE,3,2017,Female,13,308.0,4,30.8,4,30.8,3,23.1,2,15.4,5,38.5,"{'c': 1, 's': {'gender:Mean Scale Score': 308...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47730,32K562,EVERGREEN MIDDLE SCHOOL FOR URBAN EXPLORATION,All Grades,2015,Male,161,274.0,98,60.9,52,32.3,11,6.8,0,0.0,11,6.8,"{'c': 1, 's': {'gender:Mean Scale Score': 274...."
47731,32K562,EVERGREEN MIDDLE SCHOOL FOR URBAN EXPLORATION,All Grades,2016,Female,125,286.0,48,38.4,63,50.4,12,9.6,2,1.6,14,11.2,"{'c': 1, 's': {'gender:Mean Scale Score': 286...."
47732,32K562,EVERGREEN MIDDLE SCHOOL FOR URBAN EXPLORATION,All Grades,2016,Male,141,282.0,62,44.0,70,49.6,9,6.4,0,0.0,9,6.4,"{'c': 1, 's': {'gender:Mean Scale Score': 282...."
47733,32K562,EVERGREEN MIDDLE SCHOOL FOR URBAN EXPLORATION,All Grades,2017,Female,138,289.0,49,35.5,70,50.7,14,10.1,5,3.6,19,13.8,"{'c': 1, 's': {'gender:Mean Scale Score': 289...."


In [153]:
from sklearn.linear_model import Ridge
reg = Ridge(alpha=10000000).fit(genderdata.data[["Number Tested"]], genderdata.data[["Mean Scale Score"]])
print(reg.coef_)
print(reg.intercept_)

[[0.0193756]]
[297.90298616]


In [325]:
%%time
reg = Ridge(alpha=10000000).fit(crime[['ID',
 'Geographical District Code',
 'Register',
 '# Schools',
 'Major N',
 'Oth N',
 'NoCrim N',
 'Prop N',
 'Vio N',
 'AvgOfMajor N',
 'AvgOfOth N',
 'AvgOfNoCrim N',
 'AvgOfProp N',
 'AvgOfVio N']], crime[['Zip Codes']])

 

CPU times: user 4.59 ms, sys: 3.43 ms, total: 8.02 ms
Wall time: 5.78 ms


In [52]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(mean_squared_error(gender[["Mean Scale Score"]], reg.predict(gender[["Number Tested"]])))
r2_score(gender[["Mean Scale Score"]], reg.predict(gender[["Number Tested"]]))

339.90877714710166


0.009918844428457474

In [154]:
class covariance_semiring:
    # table names is to differentiate attributes with the same name but from different tables
    # attributes are the set of relatated attributes for regression
    def lift(self, row, tablename, attributes):
        e = dict()
        e['c'] = 1
        e['s'] = dict()
        for attribute in attributes:
            e['s'][tablename + ":" + attribute] = row[attribute]
        e['Q'] = dict()
        for attribute1 in attributes:
            for attribute2 in attributes:
                if tablename + ":" + attribute1 not in e['Q']:
                    e['Q'][tablename + ":" + attribute1] = dict()
                
                e['Q'][tablename + ":" + attribute1][tablename + ":" + attribute2] = row[attribute1] * row[attribute2]
        return e
    
    def add(self, e_a, e_b):
        e = dict()
        e['c'] = e_a['c'] + e_b['c']
        e['s'] = dict()
        attributes = set(e_a['s'].keys()).union(set(e_b['s'].keys()))
        for attribute in attributes:
            e['s'][attribute] = e_a['s'].get(attribute, 0) + e_b['s'].get(attribute, 0)
        e['Q'] = dict()
        for attribute1 in attributes:
            for attribute2 in attributes:
                if attribute1 not in e['Q']:
                    e['Q'][attribute1] = dict()
                e['Q'][attribute1][attribute2] = e_a['Q'].get(attribute1, {}).get(attribute2, 0)\
                                               + e_b['Q'].get(attribute1, {}).get(attribute2, 0)
        return e
    
    
    def minus(self, e_a, e_b):
        e = dict()
        e['c'] = e_a['c'] - e_b['c']
        e['s'] = dict()
        attributes = set(e_a['s'].keys()).union(set(e_b['s'].keys()))
        for attribute in attributes:
            e['s'][attribute] = e_a['s'].get(attribute, 0) - e_b['s'].get(attribute, 0)
        e['Q'] = dict()
        for attribute1 in attributes:
            for attribute2 in attributes:
                if attribute1 not in e['Q']:
                    e['Q'][attribute1] = dict()
                e['Q'][attribute1][attribute2] = e_a['Q'].get(attribute1, {}).get(attribute2, 0)\
                                               - e_b['Q'].get(attribute1, {}).get(attribute2, 0)
        return e
    
    def multiply(self, e_a, e_b):
        e = dict()
        e['c'] = e_a['c'] * e_b['c']
        e['s'] = dict()
        attributes = set(e_a['s'].keys()).union(set(e_b['s'].keys()))
        for attribute in attributes:
            e['s'][attribute] = e_a['s'].get(attribute, 0)*e_b['c'] + e_b['s'].get(attribute, 0)*e_a['c']
        e['Q'] = dict()
        for attribute1 in attributes:
            for attribute2 in attributes:
                if attribute1 not in e['Q']:
                    e['Q'][attribute1] = dict()
                e['Q'][attribute1][attribute2] = e_a['Q'].get(attribute1, {}).get(attribute2, 0)*e_b['c']\
                                               + e_b['Q'].get(attribute1, {}).get(attribute2, 0)*e_a['c']\
                                               + e_a['s'].get(attribute1, 0)*e_b['s'].get(attribute2, 0)\
                                               + e_a['s'].get(attribute2, 0)*e_b['s'].get(attribute1, 0)
        return e

def corr(cov_matrix, attr1, attr2):
    return (cov_matrix["Q"][attr1][attr2] - cov_matrix["s"][attr1]*cov_matrix["s"][attr2]/cov_matrix["c"])/\
            math.sqrt((cov_matrix["Q"][attr1][attr1] - pow(cov_matrix["s"][attr1],2)/cov_matrix["c"]) * \
                      (cov_matrix["Q"][attr2][attr2] - pow(cov_matrix["s"][attr2],2)/cov_matrix["c"]))

cov = covariance_semiring()

# return the coefficients of features and a constant 
def linear_regression(cov_matrix, features, result):
    a = np.empty([len(features) + 1, len(features) + 1])
    b = np.empty(len(features) + 1)
    
    for i in range(len(features)):
        for j in range(len(features)):
            a[i][j] = cov_matrix['Q'][features[i]][features[j]]
    
    for i in range(len(features)):
        a[i][len(features)] = cov_matrix['s'][features[i]]
        a[len(features)][i] = cov_matrix['s'][features[i]]
        b[i] = cov_matrix['Q'][result][features[i]]
    
    b[len(features)] = cov_matrix['s'][result]
    
    a[len(features)][len(features)] = cov_matrix['c']
    
    return np.linalg.solve(a, b)

# return the coefficients of features and a constant 
def ridge_linear_regression(cov_matrix, features, result, alpha):
    a = np.empty([len(features) + 1, len(features) + 1])
    b = np.empty(len(features) + 1)
    
    for i in range(len(features)):
        for j in range(len(features)):
            a[i][j] = cov_matrix['Q'][features[i]][features[j]]
        if i == j:
            a[i][i] += alpha
    
    for i in range(len(features)):
        a[i][len(features)] = cov_matrix['s'][features[i]]
        a[len(features)][i] = cov_matrix['s'][features[i]]
        b[i] = cov_matrix['Q'][result][features[i]]
    
    b[len(features)] = cov_matrix['s'][result]
    
    a[len(features)][len(features)] = cov_matrix['c']
    
    
    return np.linalg.solve(a, b)

# def square_error(cov_matrix, features, result, parameter):
#     se = cov_matrix['Q'][result][result]
    
#     for i in range(len(features)):
#         for j in range(len(features)):
#             se += parameter[i] * parameter[j] * cov_matrix['Q'][features[i]][features[j]]
    
#     for i in range(len(features)):
#         se += 2 * parameter[i] * parameter[-1] * cov_matrix['s'][features[i]]
#         se -= 2 * parameter[i] *  cov_matrix['Q'][features[i]][result]
    
#     se -= 2 * parameter[-1] * cov_matrix['s'][result]
#     se += cov_matrix['c'] * parameter[-1] * parameter[-1]
    
#     return se

# def total_sum_of_square(cov_matrix, result):
#     return cov_matrix['Q'][result][result] - cov_matrix['s'][result] * cov_matrix['s'][result] / cov_matrix['c']

# def mean_squared_error(cov_matrix, features, result, parameter):
#     return square_error(cov_matrix, features, result, parameter)/cov_matrix['c']


# def r2(cov_matrix, features, result, parameter):
#     return 1 - square_error(cov_matrix, features, result, parameter)/total_sum_of_square(cov_matrix, result)

# def adjusted_r2(cov_matrix, features, result, parameter):
#     return 1 - (cov_matrix['c']-1)*(1 - r2(cov_matrix, features, result, parameter))/(cov_matrix['c'] - len(parameter) - 1)


In [75]:
%%time
e = dict()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs


In [82]:
%%time
gender["cov"] = gender.apply(lambda row: cov.lift(row, "gender", ["Mean Scale Score", "Number Tested"]), axis=1)

CPU times: user 2.16 s, sys: 4.43 ms, total: 2.16 s
Wall time: 2.16 s


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
